## The new, NePS-based algorithms should perform at least as well as the old ones.

Test on the Hartmann6 function with (and without) fidelity

In [ ]:
import neps
from neps import algorithms
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np

# Parameters for the Hartmann 6D function
alpha = np.array([1.0, 1.2, 3.0, 3.2])
A = np.array([
    [10.0, 3.0, 17.0, 3.5, 1.7, 8.0],
    [0.05, 10.0, 17.0, 0.1, 8.0, 14.0],
    [3.0, 3.5, 1.7, 10.0, 17.0, 8.0],
    [17.0, 8.0, 0.05, 10.0, 0.1, 14.0]
])
P = 1e-4 * np.array([
    [1312, 1696, 5569, 124, 8283, 5886],
    [2329, 4135, 8307, 3736, 1004, 9991],
    [2348, 1451, 3522, 2883, 3047, 6650],
    [4047, 8828, 8732, 5743, 1091, 381]
])
# P should be divided by 10000 to match the common constants (1e-4 factor)

def hartmann6(x1, x2, x3, x4, x5, x6):
    """x must be a 6-dimensional numpy array or list-like."""
    x = np.array([x1, x2, x3, x4, x5, x6])
    r = A * (x - P)**2
    return -np.sum(alpha * np.exp(-np.sum(r, axis=1)))

def mf_hartmann6(x1, x2, x3, x4, x5, x6, fidelity=10):
    """Multi-fidelity Hartmann 6D function.
    
    fidelity: float in (1, 10), where 1 is the lowest fidelity and 10 is the highest.
    The function value is scaled by (fidelity / 10) and noise is added inversely proportional to fidelity.
    """
    if fidelity < 1.0 or fidelity > 10.0:
        raise ValueError("Fidelity must be in the range [1, 10]")
    
    base_value = hartmann6(x1, x2, x3, x4, x5, x6)
    noise = np.random.normal(0, (10 - fidelity) / 10 * 0.1)  # Noise decreases with higher fidelity
    return {"objective_to_minimize" : base_value * (fidelity / 10) + noise,
            "cost" : fidelity}

global_optimum = [0.20168952, 0.15001069, 0.47687398, 0.2753324, 0.31165163, 0.65730053]

print(mf_hartmann6(*global_optimum, fidelity=1))  # Should be approximately -3.32237
print(mf_hartmann6(*global_optimum, fidelity=5))  # Should be approximately -3.32237
print(mf_hartmann6(*global_optimum, fidelity=10))  # Should be approximately -3.32237


-0.3703557067629127
-1.7570641798509674
-3.322368011415477


Creating four search spaces: one without fidelity and three with, two of them with either good or bad priors.

In [ ]:
class HartmannSpaceFid(neps.PipelineSpace):
    x1 = neps.Float(0, 1)
    x2 = neps.Float(0, 1)
    x3 = neps.Float(0, 1)
    x4 = neps.Float(0, 1)
    x5 = neps.Float(0, 1)
    x6 = neps.Float(0, 1)
    fidelity = neps.Fidelity(neps.Integer(1, 10))

hartmann_space_fid = HartmannSpaceFid()
hartmann_space_base = hartmann_space_fid.remove("fidelity")

hartmann_space_fid_good_priors = hartmann_space_base
hartmann_space_fid_bad_priors = hartmann_space_base
for n, param in enumerate(["x1", "x2", "x3"]):
    hartmann_space_fid_good_priors = hartmann_space_fid_good_priors.add_prior(param, global_optimum[n], "medium")
    hartmann_space_fid_bad_priors = hartmann_space_fid_bad_priors.add_prior(param, 1, "medium")

# print("Hartmann 6D space with fidelity:")
# print(hartmann_space_fid)
# print("\nHartmann 6D space without fidelity:")
# print(hartmann_space)
# print("\nHartmann 6D space with fidelity and good priors:")
# print(hartmann_space_fid_good_priors)
# print("\nHartmann 6D space with fidelity and bad priors:")
# print(hartmann_space_fid_bad_priors)

We compare the following algorithms:
- Random Search (with and without priors and fidelity)
- HyperBand (with and without priors)
- PriorBand (with and without fidelity)